In [ ]:
# Import necessary libraries
# System should have the following packages installed 
# python 3.7.6, keras 2.4.6, tensorflow 2.2, pydot, 
# graphviz, keras-resnet(not needed), imbalanced -learn
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [ ]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###
    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f,f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = layers.Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    ### END CODE HERE ###
    
    return X

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

#tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = identity_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = " + str(out[0][1][1][0]))

In [ ]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(F2, (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(F3, (1, 1), strides = (1,1), name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = layers.Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    ### END CODE HERE ###
    
    return X

In [ ]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = convolutional_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = " + str(out[0][1][1][0]))

In [ ]:
def ResNet50(input_shape = (24, 24, 3), classes = 2):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', 
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D((2,2), padding="same", name='avg_pool')(X)
    
    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
ResNet50_model = ResNet50(input_shape = (24, 24, 3), classes = 2)

In [ ]:
ResNet50_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
import pandas as pd
import csv
from os import listdir
pd.options.mode.chained_assignment = None  # default='warn'
from sklearn import preprocessing
import numpy as np

In [ ]:
preprocessed_data = pd.read_csv('C:/Users/nandh/OneDrive/Desktop/IndependantStudy/temp1/preprocessed_Train.csv', sep = ',')
#preprocessed_data
X = preprocessed_data.drop('Patient_id', axis=1)
X = X.drop('SepsisLabel', axis=1)
X = X.drop('Sepsis_Shift1', axis=1)
X = X.drop('Sepsis_Shift6', axis=1)
X = X.drop('Sepsis_Shift12', axis=1)
#X
y = preprocessed_data['SepsisLabel']
#y
y_Shift1 = preprocessed_data['Sepsis_Shift1']
#y_Shift1
y_Shift6 = preprocessed_data['Sepsis_Shift6']
#y_Shift6
y_Shift12 = preprocessed_data['Sepsis_Shift12']
#y_Shift12

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= 109)

In [ ]:
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(y_test.shape))

In [ ]:
X_train = np.random.randn(X_train.shape[0], 24, 24, 3)
X_test = np.random.randn(X_test.shape[0], 24, 24, 3)
print ("X_train shape: " + str(X_train.shape))
#print ("Y_train shape: " + str(y_train.shape))
print ("X_test shape: " + str(X_test.shape))
#print ("Y_test shape: " + str(y_test.shape))

In [ ]:
# importing one hot encoder 
from sklearn.preprocessing import OneHotEncoder
# creating one hot encoder object 
onehotencoder = OneHotEncoder()
#reshape the 1-D country array to 2-D as fit_transform expects 2-D and finally fit the object 
y_train = onehotencoder.fit_transform(y_train.values.reshape(-1,1)).toarray()
#print ("Y_train shape: " + str(y_train.shape))
y_test = onehotencoder.fit_transform(y_test.values.reshape(-1,1)).toarray()
print ("Y_train shape: " + str(y_test.shape))
print ("Y_test shape: " + str(y_test.shape))

In [ ]:
ResNet50_model.fit(X_train, y_train, epochs = 10, batch_size = 32)

In [ ]:
pre = ResNet50_model.evaluate(X_test, y_test)
print ("Loss = " + str(pre[0]))
print ("Test Accuracy = " + str(pre[1]))
print(pre)

In [ ]:
pred = np.argmax(ResNet50_model.predict(X_test), axis = 1)

In [ ]:
cmat = confusion_matrix(np.argmax(y_test, axis=1), pred)
cmat

In [ ]:
TN = cmat[0,0]
TP = cmat[0,1]
FN = cmat[1,0]
FP = cmat[1,1]
print(TP,TN,FP,FN)
sensitivity = TP/(TP+FN)
print('Sensitivity : ', sensitivity )
specificity = TN/(TN+FP)
print('Specificity : ', specificity)

In [ ]:
#SMOTE
X_train_no, X_test_no, y_train_no, y_test_no = train_test_split(X, y, test_size=0.3, random_state= 109)

from imblearn.over_sampling import SMOTE
print("Before OverSampling, counts of label '1': {}".format(sum(y_train_no==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train_no==0)))

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_train_no, y_train_no.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

In [ ]:
print ("X_train shape: " + str(X_train_res.shape))
print ("Y_train shape: " + str(y_train_res.shape))
print ("X_test shape: " + str(X_test_no.shape))
print ("Y_test shape: " + str(y_test_no.shape))

In [ ]:
X_train_res = np.random.randn(X_train_res.shape[0], 24, 24, 3)
y_test_no = np.random.randn(X_test_no.shape[0], 24, 24, 3)
print ("X_train shape: " + str(X_train_res.shape))
print ("X_test shape: " + str(X_test_no.shape))

In [ ]:
# importing one hot encoder 
from sklearn.preprocessing import OneHotEncoder
# creating one hot encoder object 
onehotencoder = OneHotEncoder()
#reshape the 1-D country array to 2-D as fit_transform expects 2-D and finally fit the object 
y_train_smote = onehotencoder.fit_transform(y_train_res.reshape(-1,1)).toarray()
#print ("Y_train shape: " + str(y_train.shape))
y_test_smote = onehotencoder.fit_transform(y_test_no.reshape(-1,1)).toarray()
print ("Y_train shape: " + str(y_train_smote.shape))
print ("Y_test shape: " + str(y_test_smote.shape))

In [ ]:
pred_smote = ResNet50_model.fit(X_train_res, y_train_smote, epochs = 10, batch_size = 32)

In [ ]:
pre_smote = ResNet50_model.evaluate(X_test_no, y_test_smote)
print ("Loss = " + str(pre_smote[0]))
print ("Test Accuracy = " + str(pre_smote[1]))
print(pre_smote)

In [ ]:
pred_smote = np.argmax(ResNet50_model.predict(X_test), axis = 1)

In [ ]:
cmat_smote = confusion_matrix(np.argmax(y_test_smote, axis=1), pred_smote)
cmat_smote

In [ ]:
TN = cmat_smote[0,0]
TP = cmat_smote[0,1]
FN = cmat_smote[1,0]
FP = cmat_smote[1,1]
print(TP,TN,FP,FN)
sensitivity = TP/(TP+FN)
print('Sensitivity : ', sensitivity )
specificity = TN/(TN+FP)
print('Specificity : ', specificity)

In [ ]:
#1 hour Shift

In [ ]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y_Shift1, test_size=0.3, random_state= 109)

print ("X_train shape: " + str(X_train_1.shape))
print ("Y_train shape: " + str(y_train_1.shape))
print ("X_test shape: " + str(X_test_1.shape))
print ("Y_test shape: " + str(y_test_1.shape))

In [ ]:
X_train_1 = np.random.randn(X_train_1.shape[0], 24, 24, 3)
X_test_1 = np.random.randn(X_test_1.shape[0], 24, 24, 3)
print ("X_train shape: " + str(X_train_1.shape))
#print ("Y_train shape: " + str(y_train.shape))
print ("X_test shape: " + str(X_test_1.shape))
#print ("Y_test shape: " + str(y_test.shape))

In [ ]:
# importing one hot encoder 
from sklearn.preprocessing import OneHotEncoder
# creating one hot encoder object 
onehotencoder = OneHotEncoder()
#reshape the 1-D country array to 2-D as fit_transform expects 2-D and finally fit the object 
y_train_1hour = onehotencoder.fit_transform(y_train_1.values.reshape(-1,1)).toarray()
#print ("Y_train shape: " + str(y_train.shape))
y_test_1hour = onehotencoder.fit_transform(y_test_1.values.reshape(-1,1)).toarray()
print ("Y_train shape: " + str(y_train_1hour.shape))
print ("Y_test shape: " + str(y_test_1hour.shape))

In [ ]:
pred_1hour = ResNet50_model.fit(X_train_1, y_train_1hour, epochs = 10, batch_size = 32)

In [ ]:
pred_1h = ResNet50_model.evaluate(X_test_1, y_test_1hour)
print ("Loss = " + str(pred_1h[0]))
print ("Test Accuracy = " + str(pred_1h[1]))
print(pred_1h)

In [ ]:
pred_1hr = np.argmax(ResNet50_model.predict(X_test_1), axis = 1)

In [ ]:
cmat_1hr = confusion_matrix(np.argmax(y_test_1hour, axis=1), pred_1hr)
cmat_1hr

In [ ]:
TN = cmat_1hr[0,0]
TP = cmat_1hr[0,1]
FN = cmat_1hr[1,0]
FP = cmat_1hr[1,1]
print(TP,TN,FP,FN)
sensitivity = TP/(TP+FN)
print('Sensitivity : ', sensitivity )
specificity = TN/(TN+FP)
print('Specificity : ', specificity)

In [ ]:
#SMOTE
X_train_1hr, X_test_1hr, y_train_1hr, y_test_1hr = train_test_split(X, y_Shift1, test_size=0.3, random_state= 109)

from imblearn.over_sampling import SMOTE
print("Before OverSampling, counts of label '1': {}".format(sum(y_train_1hr==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train_1hr==0)))

sm = SMOTE(random_state=2)
X_train_res1, y_train_res1 = sm.fit_sample(X_train_1hr, y_train_1hr.ravel())

print('After1OverSampling, the shape of train_X: {}'.format(X_train_res1.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res1.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res1==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res1==0)))

In [ ]:
print ("X_train shape: " + str(X_train_res1.shape))
print ("Y_train shape: " + str(y_train_res1.shape))
print ("X_test shape: " + str(X_test_1hr.shape))
print ("Y_test shape: " + str(y_test_1hr.shape))

In [ ]:
X_train_res1h = np.random.randn(X_train_res1.shape[0], 24, 24, 3)
X_test_1h = np.random.randn(X_test_1hr.shape[0], 24, 24, 3)
print ("X_train shape: " + str(X_train_res1h.shape))
print ("X_test shape: " + str(X_test_1h.shape))

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()
y_train_smote1hr = onehotencoder.fit_transform(y_train_res1.reshape(-1,1)).toarray()
y_test_smote1hr = onehotencoder.fit_transform(y_test_1hr.values.reshape(-1,1)).toarray()
print ("Y_train shape: " + str(y_train_smote1hr.shape))
print ("Y_test shape: " + str(y_test_smote1hr.shape))

In [ ]:
pred_1hour = ResNet50_model.fit(X_train_res1h, y_train_smote1hr, epochs = 10, batch_size = 32)

In [ ]:
pred_smote_1h = ResNet50_model.evaluate(X_test_1h, y_test_smote1hr)
print ("Loss = " + str(pred_smote_1h[0]))
print ("Test Accuracy = " + str(pred_smote_1h[1]))
print(pred_smote_1h)

In [ ]:
pred_sm_1hr = np.argmax(ResNet50_model.predict(X_test_1h), axis = 1)

In [ ]:
cmat_sm_1hr = confusion_matrix(np.argmax(y_test_smote1hr, axis=1), pred_sm_1hr)
cmat_sm_1hr

In [ ]:
TN = cmat_sm_1hr[0,0]
TP = cmat_sm_1hr[0,1]
FN = cmat_sm_1hr[1,0]
FP = cmat_sm_1hr[1,1]
print(TP,TN,FP,FN)
sensitivity = TP/(TP+FN)
print('Sensitivity : ', sensitivity )
specificity = TN/(TN+FP)
print('Specificity : ', specificity)

In [ ]:
print(f1_score(np.argmax(y_test_smote1hr, axis=1), pred_sm_1hr))

In [ ]:
# 6 hour

In [ ]:
#SMOTE
X_train_6hr, X_test_6hr, y_train_6hr, y_test_6hr = train_test_split(X, y_Shift6, test_size=0.3, random_state= 109)

from imblearn.over_sampling import SMOTE
print("Before OverSampling, counts of label '1': {}".format(sum(y_train_6hr==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train_6hr==0)))

sm = SMOTE(random_state=2)
X_train_res6, y_train_res6 = sm.fit_sample(X_train_6hr, y_train_6hr.ravel())

print('After1OverSampling, the shape of train_X: {}'.format(X_train_res6.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res6.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res6==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res6==0)))

In [ ]:
print ("X_train shape: " + str(X_train_res6.shape))
print ("Y_train shape: " + str(y_train_res6.shape))
print ("X_test shape: " + str(X_test_6hr.shape))
print ("Y_test shape: " + str(y_test_6hr.shape))

In [ ]:
X_train_res6h = np.random.randn(X_train_res6.shape[0], 24, 24, 3)
X_test_6h = np.random.randn(X_test_6hr.shape[0], 24, 24, 3)
print ("X_train shape: " + str(X_train_res6h.shape))
print ("X_test shape: " + str(X_test_6h.shape))

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()
y_train_smote6hr = onehotencoder.fit_transform(y_train_res6.reshape(-1,1)).toarray()
y_test_smote6hr = onehotencoder.fit_transform(y_test_6hr.values.reshape(-1,1)).toarray()
print ("Y_train shape: " + str(y_train_smote6hr.shape))
print ("Y_test shape: " + str(y_test_smote6hr.shape))

In [ ]:
pred_6hour = ResNet50_model.fit(X_train_res6h, y_train_smote6hr, epochs = 10, batch_size = 32)

In [ ]:
pred_smote_6h = ResNet50_model.evaluate(X_test_6h, y_test_smote6hr)
print ("Loss = " + str(pred_smote_6h[0]))
print ("Test Accuracy = " + str(pred_smote_6h[1]))
print(pred_smote_6h)

In [ ]:
pred_sm_6hr = np.argmax(ResNet50_model.predict(X_test_6h), axis = 1)

In [ ]:
cmat_sm_6hr = confusion_matrix(np.argmax(y_test_smote6hr, axis=1), pred_sm_6hr,labels=[0,1])
cmat_sm_6hr

In [ ]:
TN = cmat_sm_6hr[0,0]
TP = cmat_sm_6hr[0,1]
FN = cmat_sm_6hr[1,0]
FP = cmat_sm_6hr[1,1]
print(TP,TN,FP,FN)
sensitivity = TP/(TP+FN)
print('Sensitivity : ', sensitivity )
specificity = TN/(TN+FP)
print('Specificity : ', specificity)

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(np.argmax(y_test_smote6hr, axis=1), pred_sm_6hr,labels = [1,0]))

In [ ]:
#12 hr

In [ ]:
#SMOTE
X_train_12hr, X_test_12hr, y_train_12hr, y_test_12hr = train_test_split(X, y_Shift12, test_size=0.3, random_state= 109)

from imblearn.over_sampling import SMOTE
print("Before OverSampling, counts of label '1': {}".format(sum(y_train_12hr==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train_12hr==0)))

sm = SMOTE(random_state=2)
X_train_res12, y_train_res12 = sm.fit_sample(X_train_12hr, y_train_12hr.ravel())

print('After1OverSampling, the shape of train_X: {}'.format(X_train_res12.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res12.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res12==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res12==0)))

In [ ]:
print ("X_train shape: " + str(X_train_res12.shape))
print ("Y_train shape: " + str(y_train_res12.shape))
print ("X_test shape: " + str(X_test_12hr.shape))
print ("Y_test shape: " + str(y_test_12hr.shape))

In [ ]:
X_train_res12h = np.random.randn(X_train_res12.shape[0], 24, 24, 3)
X_test_12h = np.random.randn(X_test_12hr.shape[0], 24, 24, 3)
print ("X_train shape: " + str(X_train_res12h.shape))
print ("X_test shape: " + str(X_test_12h.shape))

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()
y_train_smote12hr = onehotencoder.fit_transform(y_train_res12.reshape(-1,1)).toarray()
y_test_smote12hr = onehotencoder.fit_transform(y_test_12hr.values.reshape(-1,1)).toarray()
print ("Y_train shape: " + str(y_train_smote12hr.shape))
print ("Y_test shape: " + str(y_test_smote12hr.shape))

In [ ]:
pred_12hour = ResNet50_model.fit(X_train_res12h, y_train_smote12hr, epochs = 10, batch_size = 32)

In [ ]:
pred_smote_12h = ResNet50_model.evaluate(X_test_12h, y_test_smote12hr)
print ("Loss = " + str(pred_smote_12h[0]))
print ("Test Accuracy = " + str(pred_smote_12h[1]))
print(pred_smote_12h)

In [ ]:
pred_sm_12hr = np.argmax(ResNet50_model.predict(X_test_12h), axis = 1)

In [ ]:
cmat_sm_12hr = confusion_matrix(np.argmax(y_test_smote12hr, axis=1), pred_sm_12hr)
cmat_sm_12hr

In [ ]:
TN = cmat_sm_12hr[0,0]
TP = cmat_sm_12hr[0,1]
FN = cmat_sm_12hr[1,0]
FP = cmat_sm_12hr[1,1]
print(TP,TN,FP,FN)
sensitivity = TP/(TP+FN)
print('Sensitivity : ', sensitivity )
specificity = TN/(TN+FP)
print('Specificity : ', specificity)

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(np.argmax(y_test_smote6hr, axis=1), pred_sm_6hr))